In [0]:
%scala
import org.locationtech.jts.geom._
import org.locationtech.geomesa.spark.jts._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._

spark.withJTS
import org.locationtech.jts.geom._
import org.locationtech.geomesa.spark.jts._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._

import org.locationtech.jts.geom._
import org.locationtech.geomesa.spark.jts._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._
import org.locationtech.jts.geom._
import org.locationtech.geomesa.spark.jts._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._

In [0]:
%scala
// %scala
// import com.uber.h3core.H3Core
// import com.uber.h3core.util.GeoCoord
// import scala.collection.JavaConversions._
// import scala.collection.JavaConverters._

// object H3 extends Serializable {
//   val instance = H3Core.newInstance()
// }

// val geoToH3 = udf{ (latitude: Double, longitude: Double, resolution: Int) => 
//   H3.instance.geoToH3(latitude, longitude, resolution) 
// }
// // val h3ToGeoBoundary = udf{ (h3address: Int,res:Int) => 
// //   H3.instance.h3ToGeoBoundary(h3address) 
// // }
                  
// val polygonToH3 = udf{ (geometry: Geometry, resolution: Int) => 
//   var points: List[GeoCoord] = List()
//   var holes: List[java.util.List[GeoCoord]] = List()
//   if (geometry.getGeometryType == "Polygon") {
//     points = List(
//       geometry
//         .getCoordinates()
//         .toList
//         .map(coord => new GeoCoord(coord.y, coord.x)): _*)
//   }
//   H3.instance.polyfill(points, holes.asJava, resolution).toList 
// }

// val multiPolygonToH3 = udf{ (geometry: Geometry, resolution: Int) => 
//   var points: List[GeoCoord] = List()
//   var holes: List[java.util.List[GeoCoord]] = List()
//   if (geometry.getGeometryType == "MultiPolygon") {
//     val numGeometries = geometry.getNumGeometries()
//     if (numGeometries > 0) {
//       points = List(
//         geometry
//           .getGeometryN(0)
//           .getCoordinates()
//           .toList
//           .map(coord => new GeoCoord(coord.y, coord.x)): _* )
//     }
//     if (numGeometries > 1) {
//       holes = (1 to (numGeometries - 1)).toList.map(n => {
//         List(
//           geometry
//             .getGeometryN(n)
//             .getCoordinates()
//             .toList
//             .map(coord => new GeoCoord(coord.y, coord.x)): _*).asJava 
//       })
//     }
//   }
//   H3.instance.polyfill(points, holes.asJava, resolution).toList 
// }


In [0]:
%scala

import com.uber.h3core.H3Core
import org.apache.spark.sql.expressions.UserDefinedFunction
import org.apache.spark.sql.DataFrame

@transient lazy val h3 = new ThreadLocal[H3Core] {
    override def initialValue() = H3Core.newInstance()
  }


def convertToH3Address(xLong: Double, yLat: Double, precision: Int): String = {
    h3.get.geoToH3Address(yLat, xLong, precision)
  }

val geoToH3Address: UserDefinedFunction = udf(convertToH3Address _)
//h3.get.h3ToGeoBoundary()

def createSpatialIndexAddress(result: String,
                                XLongColumn: String,
                                YLatColumn: String,
                                precision: Int)(df: DataFrame): DataFrame = {
    df.withColumn(
      result,
      geoToH3Address(col(XLongColumn), col(YLatColumn), lit(precision)))
  }


import com.uber.h3core.H3Core
import org.apache.spark.sql.expressions.UserDefinedFunction
import org.apache.spark.sql.DataFrame
h3: ThreadLocal[com.uber.h3core.H3Core] = <lazy>
convertToH3Address: (xLong: Double, yLat: Double, precision: Int)String
geoToH3Address: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function3>,StringType,Some(List(DoubleType, DoubleType, IntegerType)))
createSpatialIndexAddress: (result: String, XLongColumn: String, YLatColumn: String, precision: Int)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame

In [0]:
%scala
// %scala

//   def geomUdf = udf { (h3Index: String) =>
//     var hexagonCoords = h3
//       .h3ToGeoBoundary(h3Index)
//       .asScala
//       .map { c => new Coordinate(c.lng, c.lat) }
//       .toArray
//     // Add start point to end of coord array, its not included in
//     // h3 output and JTS linear rings expect it.
//     hexagonCoords = hexagonCoords :+ hexagonCoords(0)
//     val polygon = Polygon(hexagonCoords)
//     polygon.reproject(LatLng, WebMercator)
//   }

In [0]:
%scala
// //h3 -->热区的那个点--->六边形
//   private def h3To6(geoCode:Long): List[GeoCoord] ={
//     val boundary: util.List[GeoCoord] = h3.h3ToGeoBoundary(geoCode)
//     JavaConverters.asScalaIteratorConverter(boundary.iterator()).asScala.toList
//   }

In [0]:
%scala
// %scala
// import com.uber.h3core.H3Core
// val h3 = H3Core.newInstance
// println(h3.h3ToGeoBoundary("8c41766410d17ff"))
// // object h3Test {
// //   val h3 = H3Core.newInstance

// //   def locationToH3(lat: Double, lon: Double, res: Int): Long = {
// //     h3.geoToH3(lat, lon, res)
// //   }

// //   def main(args: Array[String]): Unit = {
// //     val h3code1 = locationToH3(19.715031,110.563673,12)
// // //    val h3code2 = locationToH3(19.715031,110.563673,10)
// //     println("h3code1:"+h3code1);
// // //    println("h3code2:"+h3code2);

// //     println(h3.geoToH3Address(19.7157,110.5636,12))
// //     println(h3.geoToH3Address(19.7149,110.5637,12))
// //     println(h3.geoToH3Address(19.715657,110.563764,12))
// //     println(h3.geoToH3Address(19.715688,110.563662,12))
// //     println(h3.h3ToGeoBoundary("8c41766410d17ff"))
// //     println(h3.h3ToGeoBoundary("8c41766410c29ff"))
// //     println(h3.h3ToGeoBoundary("8c41766410d15ff"))
// //   }
// // }

[GeoCoord{lat=19.715598, lng=110.563742}, GeoCoord{lat=19.715698, lng=110.563754}, GeoCoord{lat=19.715762, lng=110.563670}, GeoCoord{lat=19.715726, lng=110.563572}, GeoCoord{lat=19.715626, lng=110.563560}, GeoCoord{lat=19.715562, lng=110.563644}]
import com.uber.h3core.H3Core
h3: com.uber.h3core.H3Core = com.uber.h3core.H3Core@2aa17a14

In [0]:
%scala
// %scala
// import java.util
// import scala.collection.JavaConverters
// import com.uber.h3core.util.GeoCoord

// val h3 = H3Core.newInstance
// val boundary: util.List[GeoCoord]=h3.h3ToGeoBoundary("87536bc05ffffff")
// JavaConverters.asScalaIteratorConverter(boundary.iterator()).asScala.toList


import java.util
import scala.collection.JavaConverters
import com.uber.h3core.util.GeoCoord
h3: com.uber.h3core.H3Core = com.uber.h3core.H3Core@2115dea7
boundary: java.util.List[com.uber.h3core.util.GeoCoord] = [GeoCoord{lat=25.372631, lng=51.528400}, GeoCoord{lat=25.360495, lng=51.524815}, GeoCoord{lat=25.351811, lng=51.533496}, GeoCoord{lat=25.355263, lng=51.545759}, GeoCoord{lat=25.367396, lng=51.549344}, GeoCoord{lat=25.376081, lng=51.540666}]
res18: List[com.uber.h3core.util.GeoCoord] = List(GeoCoord{lat=25.372631, lng=51.528400}, GeoCoord{lat=25.360495, lng=51.524815}, GeoCoord{lat=25.351811, lng=51.533496}, GeoCoord{lat=25.355263, lng=51.545759}, GeoCoord{lat=25.367396, lng=51.549344}, GeoCoord{lat=25.376081, lng=51.540666})

In [0]:
%scala
// %scala
// List<GeoCoord> geoCoords = h3.h3ToGeoBoundary("87536bc05ffffff");

In [0]:
%scala
// val res = 7
// val dfH3 = df.withColumn("h3index", geoToH3(col("pickup_latitude"), col("pickup_longitude"), lit(res)))
// val wktDFH3 = wktDF.withColumn("h3index", multiPolygonToH3(col("the_geom"),lit(res))).withColumn("h3index", explode($"h3index"))

In [0]:
%scala

val df_raw = spark.table("entertainer_b2c_2020.entertainer_geolocation_events")
df_raw.show()

+--------------------+-------------------+-----------+-------------------+---------+-----------+--------------------+
 id| created_on_azure|customer_id| datetime| lat| lon| session_id|
+--------------------+-------------------+-----------+-------------------+---------+-----------+--------------------+
5e672b79c32291d50...|2020-03-10 05:54:01| 9701413|2020-03-10 03:57:19| 0.00| 0.00| 317-1583816239|
5e672b79c32291d50...|2020-03-10 05:54:01| 9701413|2020-03-10 03:57:48|25.369062| 51.549784| 317-1583816239|
5e672b79c32291d50...|2020-03-10 05:54:01| 9701413|2020-03-10 03:57:19|25.369118| 51.549815| 317-1583816239|
5e672b79c32291d50...|2020-03-10 05:54:01| 2624306|2020-03-10 08:46:51|1.2899186|103.8374076|and-1583815609375...|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 04:59:50|24.470977| 54.343308| 396-1583816390|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 05:00:02|24.471564| 54.343558| 396-1583816390|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 04:59:54|24.471327| 54.343175| 396-1583816390|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 05:00:05|24.471155| 54.343282| 396-1583816390|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 04:59:50| 0.00| 0.00| 396-1583816390|
5e672b7ac32291d50...|2020-03-10 05:54:02| 2624306|2020-03-10 09:06:54|1.2899186|103.8374076|and-1583815701886...|
5e672b7ac32291d50...|2020-03-10 05:54:02| 3280541|2020-03-10 08:05:55| 1.354167| 103.760962| 300-1583813110|
5e672b7bc32291d50...|2020-03-10 05:54:03| 3280541|2020-03-10 08:13:38| 1.354167| 103.760962| 921-1583813619|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:23| 0.00| 0.00| 779-1583813724|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:23| 1.347682| 103.874787| 779-1583813724|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:39| 1.347649| 103.874781| 779-1583813724|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:26| 1.347700| 103.874794| 779-1583813724|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:39| 1.347632| 103.874778| 779-1583813724|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:24| 1.347686| 103.874784| 779-1583813724|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:30| 1.347722| 103.874791| 779-1583813724|
5e672b7bc32291d50...|2020-03-10 05:54:03| 3330640|2020-03-10 09:05:03| 1.282709| 103.846231| 340-1583816703|
+--------------------+-------------------+-----------+-------------------+---------+-----------+--------------------+
only showing top 20 rows

df_raw: org.apache.spark.sql.DataFrame = [id: string, created_on_azure: string ... 5 more fields]

In [0]:
%scala
df_raw.count()

res2: Long = 123507592

In [0]:
%scala
// from shapely.geometry.polygon import Polygon
// Polygon(h3.h3_to_geo_boundary(h3_hex_string))

In [0]:
%scala
df_raw.transform(createSpatialIndexAddress("h3","lon", "lat", 7)).show(false)

+------------------------+-------------------+-----------+-------------------+----------+---------+--------------+---------------+
id |created_on_azure |customer_id|datetime |lat |lon |session_id |h3 |
+------------------------+-------------------+-----------+-------------------+----------+---------+--------------+---------------+
5e352ec5c3229125286419c4|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:36|25.365489 |51.541508|287-1580539957|87536bc05ffffff|
5e352ec5c3229125286419c7|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:49|25.365631 |51.541166|287-1580539957|87536bc05ffffff|
5e352ec5c322910f28640cd8|2020-02-01 07:54:45|907261 |2020-02-01 07:09:46|0.00 |0.00 |512-1580540914|87754e64dffffff|
5e352ec5c3229125286419ca|2020-02-01 07:54:45|4596335 |2020-02-01 05:53:08|25.365660 |51.541146|287-1580539957|87536bc05ffffff|
5e352ec5c3229125286419cc|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:41|25.365603 |51.541179|287-1580539957|87536bc05ffffff|
5e352ec5c3229125286419cf|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:56|25.365660 |51.541150|287-1580539957|87536bc05ffffff|
5e352ec5c3229125286419d1|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:56|25.365650 |51.541142|287-1580539957|87536bc05ffffff|
5e352ec5c3229125286419d5|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:36|0.00 |0.00 |287-1580539957|87754e64dffffff|
5e352ec5c3229125286419d8|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:37|25.365484 |51.541224|287-1580539957|87536bc05ffffff|
5e352ec5c3229125286419dc|2020-02-01 07:54:45|4596335 |2020-02-01 05:52:39|25.365511 |51.541139|287-1580539957|87536bc05ffffff|
5e352ec5c322910f28640cf3|2020-02-01 07:54:45|2601360 |2020-02-01 07:03:16|25.001313 |55.155850|220-1580540596|8743a13adffffff|
5e352ec6c322910f28640cf7|2020-02-01 07:54:46|2601360 |2020-02-01 07:03:25|25.001329 |55.155833|220-1580540596|8743a13adffffff|
5e352ec6c3229125286419e6|2020-02-01 07:54:46|7092324 |2020-02-01 04:52:07|-33.831758|18.691925|306-1580539926|87ad3619cffffff|
5e352ec6c322910f28640cfa|2020-02-01 07:54:46|2601360 |2020-02-01 07:03:28|25.001344 |55.155931|220-1580540596|8743a13adffffff|
5e352ec6c3229125286419ea|2020-02-01 07:54:46|7092324 |2020-02-01 04:52:06|0.00 |0.00 |306-1580539926|87754e64dffffff|
5e352ec6c322910f28640cff|2020-02-01 07:54:46|2601360 |2020-02-01 07:03:38|25.001533 |55.155942|220-1580540596|8743a13adffffff|
5e352ec6c3229125286419ed|2020-02-01 07:54:46|7092324 |2020-02-01 04:52:06|-33.831671|18.691988|306-1580539926|87ad3619cffffff|
5e352ec6c3229125286419f1|2020-02-01 07:54:46|2917431 |2020-01-31 12:11:19|25.073741 |55.130733|238-1580472679|8743a1300ffffff|
5e352ec6c3229125286419f5|2020-02-01 07:54:46|2917431 |2020-01-31 12:11:27|25.073691 |55.130711|238-1580472679|8743a1300ffffff|
5e352ec6c3229125286419f8|2020-02-01 07:54:46|2917431 |2020-01-31 12:11:20|25.073756 |55.130724|238-1580472679|8743a1300ffffff|
+------------------------+-------------------+-----------+-------------------+----------+---------+--------------+---------------+
only showing top 20 rows

In [0]:
%scala
val df_raw_hex=df_raw.transform(createSpatialIndexAddress("h3","lon", "lat", 7))
df_raw_hex.show()

+--------------------+-------------------+-----------+-------------------+---------+-----------+--------------------+---------------+
 id| created_on_azure|customer_id| datetime| lat| lon| session_id| h3|
+--------------------+-------------------+-----------+-------------------+---------+-----------+--------------------+---------------+
5e672b79c32291d50...|2020-03-10 05:54:01| 9701413|2020-03-10 03:57:19| 0.00| 0.00| 317-1583816239|87754e64dffffff|
5e672b79c32291d50...|2020-03-10 05:54:01| 9701413|2020-03-10 03:57:48|25.369062| 51.549784| 317-1583816239|87536bc2effffff|
5e672b79c32291d50...|2020-03-10 05:54:01| 9701413|2020-03-10 03:57:19|25.369118| 51.549815| 317-1583816239|87536bc2effffff|
5e672b79c32291d50...|2020-03-10 05:54:01| 2624306|2020-03-10 08:46:51|1.2899186|103.8374076|and-1583815609375...|876520d86ffffff|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 04:59:50|24.470977| 54.343308| 396-1583816390|8743a0222ffffff|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 05:00:02|24.471564| 54.343558| 396-1583816390|8743a0222ffffff|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 04:59:54|24.471327| 54.343175| 396-1583816390|8743a0222ffffff|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 05:00:05|24.471155| 54.343282| 396-1583816390|8743a0222ffffff|
5e672b7ac32291d50...|2020-03-10 05:54:02| 60710|2020-03-10 04:59:50| 0.00| 0.00| 396-1583816390|87754e64dffffff|
5e672b7ac32291d50...|2020-03-10 05:54:02| 2624306|2020-03-10 09:06:54|1.2899186|103.8374076|and-1583815701886...|876520d86ffffff|
5e672b7ac32291d50...|2020-03-10 05:54:02| 3280541|2020-03-10 08:05:55| 1.354167| 103.760962| 300-1583813110|876520d9bffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 3280541|2020-03-10 08:13:38| 1.354167| 103.760962| 921-1583813619|876520d9bffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:23| 0.00| 0.00| 779-1583813724|87754e64dffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:23| 1.347682| 103.874787| 779-1583813724|876526365ffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:39| 1.347649| 103.874781| 779-1583813724|876526365ffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:26| 1.347700| 103.874794| 779-1583813724|876526365ffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:39| 1.347632| 103.874778| 779-1583813724|876526365ffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:24| 1.347686| 103.874784| 779-1583813724|876526365ffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 9283660|2020-03-10 08:15:30| 1.347722| 103.874791| 779-1583813724|876526365ffffff|
5e672b7bc32291d50...|2020-03-10 05:54:03| 3330640|2020-03-10 09:05:03| 1.282709| 103.846231| 340-1583816703|876520d86ffffff|
+--------------------+-------------------+-----------+-------------------+---------+-----------+--------------------+---------------+
only showing top 20 rows

df_raw_hex: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, created_on_azure: string ... 6 more fields]

In [0]:
%scala
// %sql
// drop database geo_hex_mapping;

In [0]:
%sql
Drop table if exists geo_hexmapping.ent_geoloc_hex_address_2020;


In [0]:
%scala
df_raw_hex.write.format("delta").saveAsTable("geo_hexmapping.ent_geoloc_hex_address_2020")

In [0]:
%scala
// val res = 7
// val dfH3 = df_raw.withColumn("h3index", geoToH3(col("lat"), col("lon"), lit(res)))
// dfH3.show()